# END TO END MACHINE LEARNING PROJECT EXANPLE

## Most of the time there are 8 steps you'll usually go through in ML
## 1. Look at the big Picture
## 2. Get the Data
## 3. Discover and Visusalise the data to get insights
## 4. Prepare the data for machine learning algorithm
## 5. Select a Model and train it
## 6. Fine Tune Your model
## 7. Present Your solution
## 8. Launch Monitor and maintain your system|

## Popular open data repositories
UC Irvine Machine Learning Repository

Kaggle datasets

Amazon’s AWS datasets

## Meta portals (they list open data repositories) Data Portal


 OpenDataMonit
    
 ndl

##  Other pages listing many popular open data repositoriesWikipedia’s list of Machine Learning datasets


Quora.co


The datasets subreditdditndl

In [2]:
import os
import tarfile
import urllib

In [3]:
DOWNLOAD_PATH = r"https://raw.githubusercontent.com/ageron/handsonml2/master/"

HOUSING_PATH = os.path.join("datasets", "housing")

HOUSING_URL = DOWNLOAD_PATH + r"datasets/housing/housing.tgz"

In [ ]:
def fetch_data(housing_url=HOUSING_URL)